In [46]:
import pandas as pd
import json
from ast import literal_eval
import os

# ARC

In [17]:
df = pd.read_csv('datasets/arc_challenge_train_with_prompt.csv')
df.head()


,id,question,choices,answerKey,options,prompt
0,Mercury_SC_415702,George wants to warm his hands quickly by rubb...,"{'text': array(['dry palms', 'wet palms', 'pal...",A,"['dry palms', 'wet palms', 'palms covered with...",Question: - George wants to warm his hands qui...
1,MCAS_2009_5_6516,Which of the following statements best explain...,{'text': array(['The refrigerator door is smoo...,B,"['The refrigerator door is smooth.', 'The refr...",Question: - Which of the following statements ...
2,Mercury_7233695,A fold observed in layers of sedimentary rock ...,"{'text': array(['cooling of flowing magma.', '...",B,"['cooling of flowing magma.', 'converging of c...",Question: - A fold observed in layers of sedim...
3,Mercury_7041615,Which of these do scientists offer as the most...,"{'text': array(['worldwide disease', 'global m...",D,"['worldwide disease', 'global mountain buildin...",Question: - Which of these do scientists offer...
4,Mercury_7041860,A boat is acted on by a river current flowing ...,"{'text': array(['west', 'east', 'north', 'sout...",B,"['west', 'east', 'north', 'south']",Question: - A boat is acted on by a river curr...


In [18]:
with open('datasets/arc_challenge_train_gpt4_response_parsed.json', 'r') as f:
    parsed_data = json.load(f)
len(parsed_data)

1000

In [19]:
keys = list(parsed_data.keys())
part1_keys, part2_keys, part3_keys, part4_keys = keys[:250], keys[250:500], keys[500:750], keys[750:]
len(part1_keys), len(part2_keys), len(part3_keys), len(part4_keys)

(250, 250, 250, 250)

In [20]:
key_to_answer = {'A' : 0, 'B' : 1, 'C' : 2, 'D' : 3, 'E': 4}

In [21]:
text_file_template = """Key: {}

Question: {}
Options: {}
Answer: {}

Model Response:
Concepts: {}

Relationships: {}

Model Answer: {}
"""

In [40]:
partitions = [part1_keys, part2_keys, part3_keys, part4_keys]

verfication_data = {}

for idx, part in enumerate(partitions):
    text_data = []
    for key in part:
        val = parsed_data[key]
        row = df[df['id'] == key]
        question = row['question'].values[0]
        options = row['options'].values[0]
        list_options = literal_eval(options)
        try:
            answer = list_options[key_to_answer[row['answerKey'].values[0]]]
        except:
            answer = list_options[int(row['answerKey'].values[0])-1]
        response = val['parsed_response']
        text = text_file_template.format(
            key, 
            question, 
            options, 
            answer, 
            json.dumps(response['concepts'], indent=4), 
            json.dumps(response['relationships'], indent=4), 
            json.dumps(response['final_answer'], indent=4)
        )
        text_data.append((key, text))
    verfication_data[idx] = text_data


In [41]:
len(verfication_data)

4

In [42]:
for v in verfication_data.values():
    print(len(v))

250
250
250
250


In [43]:
SAVE_PATH = "datasets/verification/arc"

In [44]:
team = ['nehal', 'dikshitha', 'muyuan', 'abhishek']

In [47]:
for idx, data in verfication_data.items():
    directory = f"{SAVE_PATH}/{team[idx]}"
    if not os.path.exists(directory):
        os.makedirs(directory)
    for i, (key, text) in enumerate(data):
        with open(f"{SAVE_PATH}/{team[idx]}/{key}.txt", "w") as file:
            file.write(text)